In [1]:
import os
import re
from collections import defaultdict
from typing import Dict, Set, Tuple
import csv


def find_unused_exports(directory: str) -> Tuple[Dict[str, Dict[str, int]], Set[str]]:
    """
    Find all exported functions/variables in .ts and .svelte files within a directory
    and track their usage throughout the codebase using simple name-based searching.

    Returns:
        - A dictionary mapping exported names to files and their usage count
        - A set of unused exported names
    """
    # Improved regex to capture only the named export, not types in signatures
    export_pattern = re.compile(r'''
        export\s+                                       # Export keyword
        (?:
            (?:function|const|let|var|class|interface|type|enum|async\s+function)  # Declaration type
            \s+([a-zA-Z_]\w*)                           # Name (capture group 1)
            |
            \{([^}]*)\}                                 # Named exports like: export { foo, bar as baz } (capture group 2)
        )
    ''', re.VERBOSE)

    # File extensions to scan
    extensions = ('.ts', '.svelte', '.js', '.jsx', '.tsx')

    # To store all exported names and their source files
    exports_map = {}  # name -> source file
    all_files = []

    # First pass: Collect all exported names
    print("Scanning for exports...")
    for root, _, files in os.walk(directory):
        for filename in files:
            if filename.endswith(extensions):
                filepath = os.path.join(root, filename)
                relative_path = os.path.relpath(filepath, directory)
                all_files.append((relative_path, filepath))

                try:
                    with open(filepath, 'r', encoding='utf-8') as file:
                        content = file.read()

                        # Process line by line to better handle export statements
                        for line_num, line in enumerate(content.splitlines(), 1):
                            # Find explicit exports
                            for match in export_pattern.finditer(line):
                                # Handle regular exports (function, const, etc.)
                                if match.group(1):
                                    name = match.group(1)
                                    exports_map[name] = relative_path
                                    print(f"Found export: {name} in {relative_path}:{line_num}")

                                # Handle named exports
                                elif match.group(2):
                                    # Split the exports and handle potential renames
                                    for item in re.split(r',\s*', match.group(2)):
                                        item = item.strip()
                                        if item:
                                            # Handle "as" renaming: "originalName as exportName"
                                            if ' as ' in item:
                                                _, export_name = item.split(' as ')
                                                export_name = export_name.strip()
                                                exports_map[export_name] = relative_path
                                                print(f"Found named export: {export_name} in {relative_path}:{line_num}")
                                            else:
                                                exports_map[item] = relative_path
                                                print(f"Found named export: {item} in {relative_path}:{line_num}")

                        # Handle default exports (separately, looking for entire lines)
                        for line_num, line in enumerate(content.splitlines(), 1):
                            default_export_match = re.search(r'export\s+default\s+([a-zA-Z_]\w*)', line)
                            if default_export_match:
                                name = default_export_match.group(1)
                                exports_map[name] = relative_path
                                print(f"Found default export: {name} in {relative_path}:{line_num}")
                except Exception as e:
                    print(f"Error reading {filepath}: {e}")

    print(f"Found {len(exports_map)} exported symbols")

    # Second pass: Count occurrences of each exported name using simple search
    print("Scanning for usage...")
    usage_by_export = defaultdict(lambda: defaultdict(int))

    for relative_path, filepath in all_files:
        try:
            with open(filepath, 'r', encoding='utf-8') as file:
                content = file.read()

                # Remove comments for accurate counting
                code_without_comments = re.sub(r'//.*?$|/\*[\s\S]*?\*/', '', content, flags=re.MULTILINE)

                for export_name, source_file in exports_map.items():
                    # Count all occurrences of the name, regardless of context
                    # Simple search pattern - looking for the word with boundaries
                    pattern = r'\b' + re.escape(export_name) + r'\b'
                    matches = re.findall(pattern, code_without_comments)

                    if matches:
                        usage_by_export[export_name][relative_path] = len(matches)
        except Exception as e:
            print(f"Error reading {filepath}: {e}")

    # Find unused exports (exports with no usage or only used in their source file)
    unused_exports = set()
    for export_name, usages in usage_by_export.items():
        source_file = exports_map.get(export_name)

        # If only used in source file or not used at all
        if not usages or (len(usages) == 1 and source_file in usages):
            unused_exports.add(export_name)

        # If used exactly once in source file (likely just the definition)
        elif len(usages) > 1 or (len(usages) == 1 and source_file not in usages):
            # Used in other files, so not unused
            pass
        else:
            unused_exports.add(export_name)

    return usage_by_export, unused_exports, exports_map


def write_results_to_csv(usage_data, unused_exports, exports_map, output_file="unused_exports_report.csv"):
    """Write the results to a CSV file."""
    with open(output_file, 'w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(["Export Name", "Source File", "Total Occurrences", "Unused?", "Found In Files"])

        for export_name in sorted(exports_map.keys()):
            source_file = exports_map.get(export_name, "Unknown")
            usages = usage_data.get(export_name, {})
            total_uses = sum(usages.values())
            is_unused = "YES" if export_name in unused_exports else "NO"
            used_in = ", ".join(usages.keys()) if usages else "None"

            writer.writerow([export_name, source_file, total_uses, is_unused, used_in])

    print(f"Results written to {output_file}")


In [2]:
# Get project directory from user (default to current directory)
directory = r"C:\Users\harold.noble\Desktop\RIC\app\frontend"
usage_data, unused_exports, exports_map = find_unused_exports(directory)

print(f"\nFound {len(unused_exports)} unused exports:")
for name in sorted(unused_exports):
    source = exports_map.get(name, "Unknown source")
    print(f"- {name} (defined in {source})")

write_results_to_csv(usage_data, unused_exports, exports_map)

Scanning for exports...
Found default export: config in config\svelte.config.js:91
Found default export: defineConfig in config\vite.config.ts:13
Found export: APP_NAME in src\lib\constants.ts:6
Found export: WEBUI_HOSTNAME in src\lib\constants.ts:11
Found export: WEBUI_BASE_URL in src\lib\constants.ts:16
Found export: WEBUI_API_BASE_URL in src\lib\constants.ts:21
Found export: OLLAMA_API_BASE_URL in src\lib\constants.ts:26
Found export: OPENAI_API_BASE_URL in src\lib\constants.ts:31
Found export: AUDIO_API_BASE_URL in src\lib\constants.ts:36
Found export: IMAGES_API_BASE_URL in src\lib\constants.ts:41
Found export: RETRIEVAL_API_BASE_URL in src\lib\constants.ts:46
Found export: WEBUI_VERSION in src\lib\constants.ts:51
Found export: WEBUI_BUILD_HASH in src\lib\constants.ts:56
Found export: PASTED_TEXT_CHARACTER_LIMIT in src\lib\constants.ts:61
Found default export: dayjs in src\lib\dayjs.js:7
Found export: getModels in src\lib\apis\index.ts:4
Found export: chatCompleted in src\lib\apis